<a href="https://colab.research.google.com/github/Tilli-simgame/SG-CompetitionNators/blob/main/koulukisojen_muotoonlaittonaattori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kilpailutulosten hakutyökalu

Tämä työkalu hakee kilpailutuloksia annetun ratsastajan ja VRL-numeron perusteella.

## Käyttöohjeet:
1. Aja solut järjestyksessä ylhäältä alas (Shift + Enter tai play-nappi solun vasemmalla puolella)
2. Ensimmäinen solu asentaa tarvittavat kirjastot
3. Syötä kilpailujen URL-osoitteet ja haettavan ratsastajan tiedot niille varattuihin soluihin
4. Aja viimeinen solu tulosten hakemiseksi

## Huomioitavaa:
- URL-osoitteiden tulee olla url osoitteita, sisällytä siis myös "http(s)://" osuus.
- Ratsastajan nimen ja VRL-numeron tulee olla täsmälleen oikeassa muodossa (esim. "Tilli (VRL-00406)")

# 1. Kirjastojen asennus

Suorita tämä solu ensimmäisenä. Se asentaa tarvittavat Python-kirjastot.

In [3]:
!pip install requests beautifulsoup4 pandas

# 2. Python-kirjastojen tuonti

Tämä solu lataa tarvittavat työkalut käyttöön.

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

# 3. Hakutoiminnot

Tämä solu sisältää ohjelman toiminnallisuuden. Suorita solu, mutta koodia ei tarvitse muokata.

In [5]:
def extract_event_details(text):
    # Try different patterns for date
    date_patterns = [
        r'Kilpailupäivä\s*\*\*([^*]+?)\*\*',
        r'Kilpailupäivä\s*([0-9]{1,2}\.[0-9]{1,2}\s*[0-9]{4})',
        r'Kilpailupäivä\s+([^\n,]+)',
    ]

    competition_date = ""
    for pattern in date_patterns:
        date_match = re.search(pattern, text, re.IGNORECASE)
        if date_match:
            competition_date = date_match.group(1).strip()
            break

    # Try different patterns for location
    location_patterns = [
        r'Järjestyspaikka\s*\*\*([^*]+?)\*\*',
        r'Järjestyspaikka\s+([^\n,]+)',
        r'Järjestyspaikka\s*([^,\n]+)'
    ]

    location = ""
    for pattern in location_patterns:
        location_match = re.search(pattern, text, re.IGNORECASE)
        if location_match:
            location = location_match.group(1).strip()
            # Remove any remaining ** if present
            location = location.replace('*', '').strip()
            break

    return competition_date, location

def extract_class_participants(text):
    class_participants = {}
    in_class_list = False
    lines = text.split('\n')

    for line in lines:
        line = line.strip()

        # Start capturing after "luokat"
        if line.lower() == 'luokat':
            in_class_list = True
            continue

        # Stop when we hit "tulokset"
        if line.lower() == 'tulokset':
            break

        if in_class_list and line:
            # Match pattern like "1. Helppo A 34/50" where 34 is current participants
            match = re.match(r'(\d+)\.\s+(.*?)\s+(\d+)/\d+', line)
            if match:
                class_num = match.group(1)
                class_name = match.group(2)
                current_participants = int(match.group(3))  # Using the first number
                class_key = f"{class_num}. {class_name}"
                class_participants[class_key] = current_participants

    return class_participants

def search_competition_results(content, search_term):
    try:
        # Create soup from content
        soup = BeautifulSoup(content, 'html.parser')
        text_content = soup.get_text()

        # Extract event details from the full text
        competition_date, location = extract_event_details(text_content)

        # Extract class participants
        class_participants = extract_class_participants(text_content)

        # Find results section
        results = []
        current_class = None
        in_results = False

        # Convert all content to text and split into lines
        lines = text_content.split('\n')

        for line in lines:
            line = line.strip()

            # Check if we've reached the results section
            if line.lower() == 'tulokset':
                in_results = True
                continue

            if not in_results:
                continue

            # Check if this is a class header
            if re.match(r'^\d+\.\s+(Helppo|Vaativa|Prix|Grand|Intermediate|KN Special)', line):
                current_class = line.split(' ', 1)[1] if ' ' in line else line  # Remove number from class name
                class_number = line.split('.')[0] + '.' if '.' in line else ''
                current_class = f"{class_number} {current_class}"
            # Check if this contains the search term
            elif search_term in line and current_class:
                placement_match = re.match(r'^\d+\.', line)
                placement = placement_match.group().strip('.') if placement_match else ''

                # Extract horse name without VRL number
                horse_match = re.search(r'\) - ([^V]+)VH', line)
                horse_name = horse_match.group(1).strip() if horse_match else ''

                # Extract VH number
                vh_match = re.search(r'(VH\d+-\d+-\d+)', line)
                vh_number = vh_match.group(1) if vh_match else ''

                # Extract class name and get total participants
                total_participants = class_participants.get(current_class, 0)

                # Class level sorting
                class_levels = {
                    'Helppo D': 1,
                    'Helppo C': 2,
                    'Helppo B': 3,
                    'KN Special': 4,
                    'Helppo A': 5,
                    'Vaativa B': 6,
                    'Vaativa A': 7,
                    'Prix St. Georges': 8,
                    'Intermediate I': 9,
                    'Intermediate II': 10,
                    'Grand Prix': 11
                }

                # Get class name without number for level lookup
                class_name_only = ' '.join(current_class.split(' ')[1:]) if ' ' in current_class else current_class
                class_level = class_levels.get(class_name_only, 0)

                results.append({
                    'class': class_name_only,
                    'class_number': int(current_class.split('.')[0]) if '.' in current_class else 999,
                    'class_level': class_level,
                    'placement': int(placement) if placement else 999,
                    'total_participants': total_participants,
                    'horse_name': horse_name,
                    'vh_number': vh_number,
                    'full_result': line,
                    'competition_date': competition_date,
                    'location': location
                })

        return results

    except Exception as e:
        print(f"Error processing content: {str(e)}")
        return []

def display_results_by_horse(results, save_to_file=False):
    if not results:
        print("No results found.")
        return

    # Create a pandas DataFrame
    df = pd.DataFrame(results)

    # Sort by horse name, class level, and placement
    df = df.sort_values(by=['horse_name', 'class_level', 'class_number', 'placement'])

    # Group by horse
    grouped = df.groupby(['horse_name', 'vh_number'])

    # Display results for each horse
    for (horse_name, vh_number), group in grouped:
        if not horse_name:  # Skip if no horse name found
            continue

        print(f"\n{horse_name} {vh_number}:")  # Added space between name and VH number

        # Sort entries by class level and placement
        sorted_entries = group.sort_values(by=['class_level', 'class_number', 'placement'])

        # Display each placement separately
        for _, row in sorted_entries.iterrows():
            if row['placement'] != 999:
                print(f"{row['competition_date']} - {row['location']} - {row['class']} - {row['placement']}/{row['total_participants']}")

    # Save to CSV if requested
    if save_to_file:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'competition_results_{timestamp}.csv'
        df.to_csv(filename, index=False)
        print(f"\nResults saved to {filename}")

def process_multiple_urls(urls, search_term):
    all_results = []

    for url in urls:
        try:
            print(f"\nProcessing {url}...")
            response = requests.get(url)
            response.encoding = 'utf-8'
            results = search_competition_results(response.text, search_term)
            all_results.extend(results)
            print(f"Found {len(results)} results")
        except Exception as e:
            print(f"Error processing {url}: {str(e)}")

    return all_results

# 4. Syötetiedot

Muokkaa alla olevat tiedot:
- Lisää kilpailujen URL-osoitteet urls-listaan
- Kirjoita search_term-kohtaan haettavan ratsastajan nimi tai VRL-numero täsmälleen oikeassa muodossa

Esimerkki:
- URL: https://harakkasyndrooma.net/n/krj/361.html
- Ratsastajan tiedot: "Tilli (VRL-00406)"

In [6]:
# Lisää kilpailujen URL-osoitteet tähän, yksi per rivi
urls = [
    "https://harakkasyndrooma.net/n/krj/361.html",
    "https://harakkasyndrooma.net/n/krj/362.html",
    "https://harakkasyndrooma.net/n/krj/431.html",
    "https://harakkasyndrooma.net/n/krj/331.html" # Poista # merkki rivin alusta ottaaksesi URL käyttöön
]

# Kirjoita ratsastajan nimi tai VRL-numero
search_term = "corbin. (VRL-06467)"  # Muuta tämä

# 5. Tulosten haku

Suorita tämä solu hakeaksesi tulokset.
- Tulokset näytetään ruudulla
- CSV-tiedosto tallennetaan automaattisesti

In [7]:
# Hae ja näytä tulokset
results = process_multiple_urls(urls, search_term)
display_results_by_horse(results, save_to_file=True)


Processing https://harakkasyndrooma.net/n/krj/361.html...
Found 65 results

Processing https://harakkasyndrooma.net/n/krj/362.html...
Found 65 results

Processing https://harakkasyndrooma.net/n/krj/431.html...
Found 25 results

Processing https://harakkasyndrooma.net/n/krj/331.html...
Found 26 results

Beaudesert SIN VH24-031-0083:
01.12 2024 - Nessinjärven kilpailukeskus - Vaativa A - 21/26
01.12 2024 - Nessinjärven kilpailukeskus - Vaativa A - 18/26
12.01 2025 - Nessinjärven kilpailukeskus - Vaativa A - 17/35
11.01 2025 - Nessinjärven kilpailukeskus - Vaativa A - 28/35
12.01 2025 - Nessinjärven kilpailukeskus - Vaativa A - 7/38
11.01 2025 - Nessinjärven kilpailukeskus - Vaativa A - 8/38

Bombay Rest VH03-021-9793:
11.01 2025 - Nessinjärven kilpailukeskus - Grand Prix - 5/41
12.01 2025 - Nessinjärven kilpailukeskus - Grand Prix - 15/41
11.01 2025 - Nessinjärven kilpailukeskus - Grand Prix - 26/39
12.01 2025 - Nessinjärven kilpailukeskus - Grand Prix - 30/39

Bravo O' Polo VH24-044-00